# Part I: Transform Toronto Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = df[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
#cleanup column names
df.rename(columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'}, inplace=True)

#delete first row in data
df.drop(df.index[[0]], inplace=True)

#exclude rows where Borough is "Not assigned"
df = df[df.Borough != 'Not assigned']

#assign missing neighborhoods with Borough's name
df.Neighborhood.replace("Not assigned",df.Borough,inplace=True)

#combine neighborhood data having same postal codes
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
df.shape

(103, 3)

# Part II: Add Coordinates to Toronto Data

In [5]:
#read coordinates csv file

latlong = pd.read_csv("http://cocl.us/Geospatial_data")
latlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#join borough/neighborhood data with coordinates based on postal code
toronto = pd.merge(df, latlong, on='PostalCode')
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [7]:
toronto.shape

(103, 5)

# Part III: Create Toronto Map and Cluster Data

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.18 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.68 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.62 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.19 MB/s
vincent-0.4.4- 100% |###################

#### A. Create a map of Toronto with neighborhoods superimposed on top

In [9]:
#Use geopy library to get the latitude and longitude values of Toronto

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'QDNSA5AF0V4GHNCOVLLT0LDWXBKUIWZIDG0MW2UPDVOXK1HJ' # your Foursquare ID
CLIENT_SECRET = 'LFWAWILOU43MYDKEO2QK5AMWUZXDZAPJJ35G11XY4JKHEYRE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### B. Use Foursquare API to find top venues in Toronto

In [57]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [58]:
print (nearby_venues)


                        name                     categories        lat  \
0         Images Salon & Spa                            Spa  43.802283   
1             Caribbean Wave           Caribbean Restaurant  43.798558   
2                    Wendy's           Fast Food Restaurant  43.802008   
3                   Harvey's           Fast Food Restaurant  43.800106   
4                    Wendy's           Fast Food Restaurant  43.807448   
5                Tim Hortons                    Coffee Shop  43.802000   
6        Staples Morningside  Paper / Office Supplies Store  43.800285   
7                 Lee Valley                     Hobby Shop  43.803161   
8         Bus Stop: 85 & 116                    Bus Station  43.802198   
9         Tim Hortons / Esso                    Coffee Shop  43.801660   
10                   Mr Jerk             African Restaurant  43.801262   
11  Charley's Exotic Cuisine             Chinese Restaurant  43.800982   
12         MR. GREEK Express          

In [59]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [60]:
print(toronto_venues.shape)
toronto_venues.head()

(4848, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
2,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Harvey's,43.800106,-79.198258,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant


In [61]:
#Check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,49,49,49,49,49,49
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",29,29,29,29,29,29
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",15,15,15,15,15,15
"Alderwood, Long Branch",24,24,24,24,24,24
"Bathurst Manor, Downsview North, Wilson Heights",24,24,24,24,24,24
Bayview Village,13,13,13,13,13,13
"Bedford Park, Lawrence Manor East",41,41,41,41,41,41
Berczy Park,100,100,100,100,100,100


In [62]:
#determine number of unique categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 335 uniques categories.


#### C. Analyze Each Neighborhood

In [63]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke

In [64]:
toronto_onehot.shape

(4848, 335)

In [65]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke

In [66]:
toronto_grouped.shape

(102, 335)

In [67]:
#Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.07
1                Hotel  0.05
2  American Restaurant  0.04
3          Coffee Shop  0.04
4              Theater  0.04


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.14
1       Shopping Mall  0.06
2              Bakery  0.04
3         Pizza Place  0.04
4         Supermarket  0.04


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.17
1                Park  0.07
2              Bakery  0.07
3         Pizza Place  0.07
4            Pharmacy  0.07


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.20
1          Pizza Place  0.20
2                 Park  0.07
3          Coffee Shop  0.07
4  Fried Chicken Joint  0.07


----Alderwood, Long Branch----
               venue  freq
0     Discount S

In [68]:
#Save data into new dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Theater,Coffee Shop,American Restaurant,Japanese Restaurant,Restaurant,Concert Hall,Sushi Restaurant,Cosmetics Shop
1,Agincourt,Chinese Restaurant,Shopping Mall,Bakery,Pizza Place,Supermarket,Caribbean Restaurant,Coffee Shop,Bank,Discount Store,Japanese Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,Pizza Place,Bakery,Pharmacy,Park,Korean Restaurant,Dessert Shop,Coffee Shop,Noodle House,Caribbean Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Coffee Shop,Bus Line,Fast Food Restaurant,Sandwich Place,Pharmacy,Hardware Store,Fried Chicken Joint,Park
4,"Alderwood, Long Branch",Discount Store,Pizza Place,Grocery Store,Pharmacy,Liquor Store,Sandwich Place,Park,Intersection,Trail,Coffee Shop


#### D. Clustering

In [71]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 4, 4, 4, 4, 0, 0, 1, 0], dtype=int32)

In [81]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)
#neighborhoods_venues_sorted.drop(columns=['ClusterLabels'], inplace=True)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#toronto_merged.ClusterLabels = toronto_merged.ClusterLabels.fillna(0).astype(int)
toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,0,Fast Food Restaurant,Coffee Shop,Sandwich Place,Bus Station,Fruit & Vegetable Store,Greek Restaurant,Paper / Office Supplies Store,Park,Gym,Hobby Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,1,Breakfast Spot,Playground,Burger Joint,Italian Restaurant,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Liquor Store,Bank,Burger Joint,Sandwich Place,Supermarket,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Chinese Restaurant,Electronics Store,Park,Fast Food Restaurant,Indian Restaurant,Pharmacy,Eastern European Restaurant,Elementary School,Empanada Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Bakery,Coffee Shop,Pharmacy,Indian Restaurant,Yoga Studio,Asian Restaurant,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Music Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Fast Food Restaurant,Sandwich Place,Bowling Alley,Train Station,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Convenience Store,Elementary School
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,0,Coffee Shop,Discount Store,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Sandwich Place,Bank,Asian Restaurant,Train Station,Light Rail Station
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,0,Coffee Shop,Bakery,Bus Line,Intersection,Diner,Fast Food Restaurant,Bank,Beer Store,Pizza Place,Mexican Restaurant
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Pizza Place,Beach,Fast Food Restaurant,Wings Joint,Sports Bar,Burger Joint,Cajun / Creole Restaurant,Furniture / Home Store,Park,Yoga Studio
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,Park,Café,Restaurant,Thai Restaurant,Diner,Gym,Skating Rink,Gym Pool,General Entertainment,Bank


#### E. Create Cluster Map

In [82]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### F. Examine Clusters

##### Cluster 1

In [84]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Coffee Shop,Sandwich Place,Bus Station,Fruit & Vegetable Store,Greek Restaurant,Paper / Office Supplies Store,Park,Gym,Hobby Shop
2,Scarborough,0,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Liquor Store,Bank,Burger Joint,Sandwich Place,Supermarket,Discount Store
5,Scarborough,0,Fast Food Restaurant,Sandwich Place,Bowling Alley,Train Station,Coffee Shop,Restaurant,Japanese Restaurant,Pizza Place,Convenience Store,Elementary School
6,Scarborough,0,Coffee Shop,Discount Store,Fast Food Restaurant,Chinese Restaurant,Grocery Store,Sandwich Place,Bank,Asian Restaurant,Train Station,Light Rail Station
7,Scarborough,0,Coffee Shop,Bakery,Bus Line,Intersection,Diner,Fast Food Restaurant,Bank,Beer Store,Pizza Place,Mexican Restaurant
8,Scarborough,0,Pizza Place,Beach,Fast Food Restaurant,Wings Joint,Sports Bar,Burger Joint,Cajun / Creole Restaurant,Furniture / Home Store,Park,Yoga Studio
9,Scarborough,0,Park,Café,Restaurant,Thai Restaurant,Diner,Gym,Skating Rink,Gym Pool,General Entertainment,Bank
10,Scarborough,0,Furniture / Home Store,Coffee Shop,Electronics Store,Asian Restaurant,Indian Restaurant,Chinese Restaurant,Bakery,Burger Joint,Fast Food Restaurant,Italian Restaurant
11,Scarborough,0,Pizza Place,Middle Eastern Restaurant,Intersection,Grocery Store,Supermarket,Badminton Court,Restaurant,Bakery,Bar,Korean Restaurant
15,Scarborough,0,Chinese Restaurant,Coffee Shop,Bakery,Fast Food Restaurant,Intersection,Breakfast Spot,Shopping Mall,Electronics Store,Sandwich Place,Other Great Outdoors


##### Cluster 2

In [85]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1,Breakfast Spot,Playground,Burger Joint,Italian Restaurant,Farmers Market,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit
21,North York,1,Korean Restaurant,Café,Pizza Place,Coffee Shop,Trail,Park,Electronics Store,Bank,Fried Chicken Joint,Shopping Mall
22,North York,1,Japanese Restaurant,Coffee Shop,Ramen Restaurant,Korean Restaurant,Pizza Place,Bubble Tea Shop,Sandwich Place,Café,Restaurant,Sushi Restaurant
30,North York,1,Coffee Shop,Electronics Store,Turkish Restaurant,Liquor Store,Café,Gym,Latin American Restaurant,Italian Restaurant,Park,Vietnamese Restaurant
34,North York,1,Coffee Shop,Portuguese Restaurant,Hockey Arena,Golf Course,Pizza Place,Lounge,Men's Store,Park,Athletics & Sports,Café
37,East Toronto,1,Coffee Shop,Pub,Pizza Place,Beach,Bakery,Bar,Sandwich Place,Breakfast Spot,Japanese Restaurant,Tea Room
39,East York,1,Coffee Shop,Indian Restaurant,Grocery Store,Supermarket,Afghan Restaurant,Pizza Place,Brewery,Turkish Restaurant,Burger Joint,Smoothie Shop
40,East York,1,Coffee Shop,Café,Greek Restaurant,Pizza Place,Park,Fast Food Restaurant,Ethiopian Restaurant,Pharmacy,Bakery,Breakfast Spot
41,East Toronto,1,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Pizza Place,Ice Cream Shop,Spa,Yoga Studio,Sandwich Place
42,East Toronto,1,Indian Restaurant,Coffee Shop,Café,Beach,Burger Joint,Pizza Place,Restaurant,Sandwich Place,Fast Food Restaurant,Butcher


##### Cluster 3

In [86]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,2,Park,Pool,Yoga Studio,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space


##### Cluster 4

In [87]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,3,Vietnamese Restaurant,Baseball Field,Restaurant,Yoga Studio,Farm,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space


##### Cluster 5

In [88]:
toronto_merged.loc[toronto_merged['ClusterLabels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4,Coffee Shop,Chinese Restaurant,Electronics Store,Park,Fast Food Restaurant,Indian Restaurant,Pharmacy,Eastern European Restaurant,Elementary School,Empanada Restaurant
4,Scarborough,4,Bakery,Coffee Shop,Pharmacy,Indian Restaurant,Yoga Studio,Asian Restaurant,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Music Store
12,Scarborough,4,Chinese Restaurant,Shopping Mall,Bakery,Pizza Place,Supermarket,Caribbean Restaurant,Coffee Shop,Bank,Discount Store,Japanese Restaurant
13,Scarborough,4,Coffee Shop,Pharmacy,Pizza Place,Fast Food Restaurant,Chinese Restaurant,Sandwich Place,Park,Discount Store,Thai Restaurant,Italian Restaurant
14,Scarborough,4,Chinese Restaurant,Pizza Place,Bakery,Pharmacy,Park,Korean Restaurant,Dessert Shop,Coffee Shop,Noodle House,Caribbean Restaurant
17,North York,4,Pharmacy,Coffee Shop,Park,Pizza Place,Residential Building (Apartment / Condo),Convenience Store,Shopping Mall,Chinese Restaurant,Sandwich Place,Korean Restaurant
24,North York,4,Pharmacy,Convenience Store,Bus Line,Shopping Mall,Grocery Store,Butcher,Bakery,Coffee Shop,Pizza Place,Eastern European Restaurant
25,North York,4,Park,Convenience Store,Bus Stop,Pharmacy,Shopping Mall,Pizza Place,Supermarket,Tennis Court,Chinese Restaurant,Caribbean Restaurant
28,North York,4,Coffee Shop,Pizza Place,Deli / Bodega,Bridal Shop,Community Center,Ski Chalet,Ski Area,Shopping Mall,Fried Chicken Joint,Frozen Yogurt Shop
31,North York,4,Bank,Pizza Place,Park,Vietnamese Restaurant,Moving Target,Grocery Store,Coffee Shop,Exhibit,Eastern European Restaurant,Electronics Store


#### G. Cluster Observations

1. General - All areas have coffee shops as a top 5 common venues
2. Cluster 1 - Central Toronto - Fast Food, Pizza/Italian, Some Chinese/Japanese Food,
3. Cluster 2 - Central Toronto - Hotels, Cafes, Theatres, and large variety of ethnic food (Italian, Korean, Chinese, Indian, etc.)
4. Cluster 3/4 - Only one neighborhood in each cluster, more active areas with pools, parks and yoga, could be similar to Cluster 2
5. Cluster 5 - Further out from downtown with more big box stores (discount, electronics, etc), golf courses